In [9]:
import numpy as np
import open3d as o3d


# Load point clouds
pc_app = o3d.io.read_point_cloud("app.ply")
pc_gt = o3d.io.read_point_cloud("ground_truth.ply")
pc_app_voxels = pc_app.voxel_down_sample(voxel_size=0.05)
pc_gt_voxels = pc_gt.voxel_down_sample(voxel_size=0.05)
print(len(pc_app_voxels.points))

# Estimate normals
pc_app_voxels.estimate_normals()
pc_gt_voxels.estimate_normals()

# Compute FPFH features
fpfh_app = o3d.pipelines.registration.compute_fpfh_feature(
    pc_app_voxels, o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=100))
fpfh_gt = o3d.pipelines.registration.compute_fpfh_feature(
    pc_gt_voxels, o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=100))


o3d.visualization.draw_geometries([pc_app])


36897
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [10]:

# Perform global registration
result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source=pc_app_voxels,
    target=pc_gt_voxels,
    source_feature=fpfh_app,
    target_feature=fpfh_gt,
    mutual_filter=False,
    max_correspondence_distance=1,
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
    ransac_n=3,
    checkers=[o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
              o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(0.1)],
    criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500)
)

# pc_app.transform(result.transformation)
refined_result = o3d.pipelines.registration.registration_icp(
    pc_app_voxels, pc_gt_voxels, 0.3,
    result.transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(refined_result.transformation)

[[ 0.68164623 -0.39423045 -0.61639336  0.56370984]
 [-0.64722107  0.06804769 -0.75925911 -0.13980757]
 [ 0.3412672   0.91648888 -0.2087698  -0.10821887]
 [ 0.          0.          0.          1.        ]]


In [11]:

print(result.transformation)
trans_pc_app = pc_app.__copy__()
trans_pc_app.transform(refined_result.transformation)
#
# Compute Hausdorff distance
distances = trans_pc_app.compute_point_cloud_distance(pc_gt)
hausdorff_distance = max(distances)
mean_distance = np.mean(distances)

print(f"Hausdorff Distance: {hausdorff_distance}")
print(f"Mean Distance: {mean_distance}") # On average, the app is about 3% off from the ground truth
print(f"MSE: {np.mean(np.asarray(distances) ** 2)}")

[[ 0.68012803 -0.412001   -0.60636709  0.55820123]
 [-0.65888651  0.01908023 -0.75200034 -0.01063642]
 [ 0.32139451  0.9109836  -0.2584849  -0.09458848]
 [ 0.          0.          0.          1.        ]]
Hausdorff Distance: 0.5107116958138777
Mean Distance: 0.08967352558618975
MSE: 0.016314065162083828


In [12]:
# Visualize
o3d.visualization.draw_geometries([trans_pc_app, pc_gt])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [6]:
from scipy.spatial import cKDTree


def uniform_sample(point_cloud, num_samples):
    """
    Uniformly sample a fixed number of points from a point cloud.
    """
    points = np.asarray(point_cloud.points)
    if len(points) < num_samples:
        raise ValueError("Point cloud has fewer points than the number of samples requested.")
    indices = np.random.choice(len(points), num_samples, replace=False)
    return points[indices]

def compute_mse_uniform_sampling(source_cloud, target_cloud, num_samples):
    """
    Compute MSE between two point clouds using uniform sampling.
    """
    # Uniformly sample points from both clouds
    source_samples = uniform_sample(source_cloud, num_samples)
    target_samples = uniform_sample(target_cloud, num_samples)

    # Use KDTree for efficient nearest-neighbor search
    target_tree = cKDTree(target_samples)
    distances, _ = target_tree.query(source_samples)

    # Compute MSE as the mean of squared distances
    mse = np.mean(distances ** 2)
    return mse

# Compute MSE with uniform sampling
num_samples = min(len(trans_pc_app.points), len(pc_gt.points)) - 1000  # Adjust the number of samples as needed
try:
    mse = compute_mse_uniform_sampling(trans_pc_app, pc_gt, num_samples)
    print(f"Mean Squared Error (MSE) with Uniform Sampling: {mse}")
except ValueError as e:
    print(f"Error: {e}")

Mean Squared Error (MSE) with Uniform Sampling: 0.012234857119167249


In [7]:
print(len(pc_gt.points))

599653
